In [4]:
# disperse
from ecc import PrivateKey
from helper import decode_base58, p2pkh_script, SIGHASH_ALL
from script import Script
from tx import TxIn, TxOut, Tx
import requests

secret = 61740721216174072121
priv = PrivateKey(secret=secret)
my_address = priv.point.address(testnet=True)
target_amount = 0.1
fee = 0.005

response = requests.get('https://testnet.blockexplorer.com/api/addr/{}/utxo'.format(my_address))
utxos = response.json()

student_addresses = [

]
# initialize inputs
tx_ins = []
total = 0
for utxo in utxos:
    prev_tx = bytes.fromhex(utxo['txid'])
    prev_index = utxo['vout']
    total += utxo['amount']
    # create a new tx input
    tx_ins.append(TxIn(
            prev_tx=prev_tx,
            prev_index=prev_index,
            script_sig=b'',
            sequence=0xffffffff,
        ))

# initialize outputs
tx_outs = []

for target_address in student_addresses:
    # decode the hash160 from the target address
    h160 = decode_base58(target_address)
    # convert hash160 to p2pkh script
    script_pubkey = p2pkh_script(h160)
    # convert target amount to satoshis (multiply by 100 million)
    target_satoshis = int(target_amount*100000000)
    # create a new tx output for target
    tx_outs.append(TxOut(
        amount=target_satoshis,
        script_pubkey=script_pubkey,
    ))

change_amount = total - target_amount*len(student_addresses) - fee

h160 = decode_base58(my_address)
script_pubkey = p2pkh_script(h160)
change_satoshis = int(change_amount*100000000)
tx_outs.append(TxOut(amount=change_satoshis, script_pubkey=script_pubkey))

# create the transaction
tx_obj = Tx(version=1, tx_ins=tx_ins, tx_outs=tx_outs, locktime=0, testnet=True)

# now sign the 0th input with the private key using SIGHASH_ALL using sign_input
for i in range(len(tx_obj.tx_ins)):
    tx_obj.sign_input(i, priv, SIGHASH_ALL)

# SANITY CHECK: fee is reasonable
if tx_obj.fee(testnet=True) > 0.05*100000000 or tx_obj.fee(testnet=True) <= 0:
    raise RuntimeError('Check that the change amount is reasonable. Fee is {}'.format(tx_obj.fee()))

# serialize and hex()
print(tx_obj.serialize().hex())

0100000010b7d09be0d47cef6e87fedd86002f36231ff9d756afea5c60c9e0214252fdc712000000006b483045022100a9b84e58276213fe77508c02d708bb150bee1370ed6108697baf4dd8ab6273e902206c9c1d388609e5d8a0fe49b183ec366e149d20ec0718c21fa646547ee633a454012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffffbce9de71827ccaabd99afe5eb68d05b74c989a56f1ba6e8447f7112e18b887ad060000006b483045022100d85bb50c9f9bfc1932ae26698431ffe8f04713f107a1fc1690c3649957090c1802206e23ef8392ac30248575930dd05ae8915dfc9330fe1a5fa09d633b5edf3d0d26012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffffbce9de71827ccaabd99afe5eb68d05b74c989a56f1ba6e8447f7112e18b887ad040000006a47304402201171ce8e981dda7715be340d20038dc1d0f3ea2c2189ba9d567e92fbd7351999022057c25fb7bc0a191c90c7e645c1a054f1a80ca9674a0b40de0cac33054a560c3f012103f96f3a1efd31e1a8d7078118ee56bff7355d58907ce0f865f5f0b3dbe34e55beffffffffbce9de71827ccaabd99afe5eb68d05b74c989a56f1ba6e8447f7112e18b887ad020000006b4830450221009fa1067633e15665c9